In [18]:
from bqplot import Tooltip, AlbersUSA, OrdinalColorScale, ColorAxis, ColorScale
import pandas as pd
import numpy as np
from bqplot import pyplot as plt

In [19]:
import requests
URL = 'https://covidtracking.com/api/states'
r = requests.get(URL)
import json
covid = pd.read_json(json.dumps(r.json()))
covid.index = covid['state']
deaths = covid['death']
tests = covid['totalTestResults']

# merge deaths with FIPS

FIPS = pd.read_csv('states.csv')
FIPS.index = FIPS['USPS']
FIPS
FIPS['deaths'] = deaths
FIPS['tests'] = tests

# create a series of populations
# create a column of normalized deaths

populations = pd.read_excel('state-populations.xlsx', index_col='State')
populations = populations['population']
FIPS['populations'] = populations
FIPS['normalized_deaths'] = FIPS['deaths']/FIPS['populations']
FIPS['normalized_tests'] = FIPS['tests']/FIPS['populations']
FIPS.head()

,Name,FIPS,USPS,deaths,tests,populations,normalized_deaths,normalized_tests
USPS,,,,,,,,
AL,Alabama,1,AL,3.0,4771,4903185,6.118472e-07,0.000973
AK,Alaska,2,AK,1.0,2388,731545,1.366970e-06,0.003264
AZ,Arizona,4,AZ,13.0,8191,7278717,1.786029e-06,0.001125
AR,Arkansas,5,AR,3.0,1926,3017804,9.941003e-07,0.000638
CA,California,6,CA,78.0,21259,39512223,1.974073e-06,0.000538


In [38]:
# this uses the plt API

# get state data FIPS
sc_geo = AlbersUSA()
# color_data_county maps from FIPS to some value

color_scale = ColorScale(scheme='viridis') 

# dict with FIPS key and data as value
color_data = dict(zip(FIPS['FIPS'], FIPS['tests'])) 
color_data = dict(zip(FIPS['FIPS'], FIPS['normalized_tests'])) 
color_data = dict(zip(FIPS['FIPS'], FIPS['normalized_deaths'])) 

map_styles_state = {'color': color_data,
                    'scales': {'projection': sc_geo,
                               'color': color_scale}}

fig = plt.figure(title='Figure')
state_map = plt.geo(map_data='USStatesMap', **map_styles_state)

fig

Figure(axes=[ColorAxis(scale=ColorScale(scheme='viridis'))], fig_margin={'top': 60, 'bottom': 60, 'left': 60, …

In [22]:
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa
import bqplot as bq

color_data = dict(zip(FIPS['FIPS'], FIPS['normalized_deaths'])) 
sc_cl = bqs.ColorScale(scheme='viridis')
sc_geo = bqs.AlbersUSA(scale_factor=1200, translate=(500,400))
axis = bqa.ColorAxis(scale=sc_cl)
map_styles = {'color': color_data,
              'scales': {'projection': sc_geo, 'color': sc_cl}, 
              'colors': {'default_color': 'Grey'}}
states_map = bqm.Map(map_data=bqm.topo_load('map_data/USStatesMap.json'), **map_styles)
bq.Figure(marks=[states_map], axes=[axis], title='US States Map Example')

Figure(axes=[ColorAxis(scale=ColorScale(scheme='viridis'))], fig_margin={'top': 60, 'bottom': 60, 'left': 60, …